In [ ]:
# basic libraries

import numpy as np # linear algebra
import pandas as pd # data processing
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Split imbalanced dataset into train and test sets with stratification: #Only used in imbalanced classification problems
# This will make sure that in train and test there will be about 8% defaulters. # Remaining structure of original data.
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Making sure all numbers would not have scientific notaion (e+8 for example):
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Extra options for convenient
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# Loading current data
app_df = pd.read_csv("../input/loan-defaulter/application_data.csv")

# Loading previous data
previous = pd.read_csv("../input/loan-defaulter/previous_application.csv")

In [ ]:
null_app_df = pd.DataFrame((app_df.isnull().sum())*100/app_df.shape[0]).reset_index() #Creating new df with 2 columns. columns from app_df saved in rows, base on "coloumn name"
null_app_df.columns = ['Column Name', 'Null Values Percentage'] #Adding names to

high_null = null_app_df[null_app_df['Null Values Percentage'] > 25]

In [ ]:
#Making a new column that is the sum of all avg information about the apartment. alot of null in this data. we want to make sure we can delete all these columns
app_df['BUILDING_TOTAL_AVG'] = app_df.loc[:, 'APARTMENTS_AVG': 'NONLIVINGAREA_MEDI'].sum(axis=1)

null_list = ['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'BUILDING_TOTAL_AVG', 'TARGET']

In [ ]:
FlagDocument_list = [col for col in app_df if col.startswith('FLAG_DOC')]

In [ ]:
app_df.shape

In [ ]:
delete_col_app = high_null['Column Name'].tolist() + FlagDocument_list 
delete_col_app.remove('FLAG_DOCUMENT_3')
delete_col_app.remove('OCCUPATION_TYPE')

app_df.drop(labels=delete_col_app, axis=1, inplace=True)
app_df = app_df.drop(labels=['EXT_SOURCE_2', 'EXT_SOURCE_3'], axis=1)
app_df = app_df.drop(labels=['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE'], axis=1)

In [ ]:
#Creating new rank children column, base on total examples in data:

app_df.loc[app_df['CNT_CHILDREN'] < 1, 'CNT_CHILDREN_RANK'] = 1
app_df.loc[(app_df['CNT_CHILDREN'] > 0) & (app_df['CNT_CHILDREN'] <= 2), 'CNT_CHILDREN_RANK'] = 2
app_df.loc[(app_df['CNT_CHILDREN'] > 2) & (app_df['CNT_CHILDREN'] <= 4), 'CNT_CHILDREN_RANK'] = 3
app_df.loc[(app_df['CNT_CHILDREN'] > 4) & (app_df['CNT_CHILDREN'] <= 6), 'CNT_CHILDREN_RANK'] = 4
app_df.loc[app_df['CNT_CHILDREN'] > 6 , 'CNT_CHILDREN_RANK'] = 5

app_df.loc[app_df['CNT_FAM_MEMBERS'] < 2, 'CNT_FAM_MEMBERS_RANK'] = 1
app_df.loc[(app_df['CNT_FAM_MEMBERS'] > 1) & (app_df['CNT_FAM_MEMBERS'] <= 2), 'CNT_FAM_MEMBERS_RANK'] = 2
app_df.loc[(app_df['CNT_FAM_MEMBERS'] > 2) & (app_df['CNT_FAM_MEMBERS'] <= 4), 'CNT_FAM_MEMBERS_RANK'] = 3
app_df.loc[(app_df['CNT_FAM_MEMBERS'] > 4) & (app_df['CNT_FAM_MEMBERS'] <= 6), 'CNT_FAM_MEMBERS_RANK'] = 4
app_df.loc[app_df['CNT_FAM_MEMBERS'] > 6, 'CNT_FAM_MEMBERS_RANK'] = 5

In [ ]:
# Saving all columns starting with 'DAYS' since those are the columns we want to change
minus_col = [col for col in app_df if col.startswith('DAYS')]

# Applying abs() function to all columns starting with 'DAYS'. 
app_df[minus_col]= abs(app_df[minus_col])

# Changing relevant columns to years. (only 2 at the moment(!))
app_df['Age'] = (app_df['DAYS_BIRTH'] / 365).round()
app_df['Years Employed'] = app_df['DAYS_EMPLOYED'] / 365 


In [ ]:
#Pensioners to 25
app_df['Years Employed'] = app_df["Years Employed"].mask((app_df["Years Employed"] > 900) & (app_df['NAME_INCOME_TYPE'] == 'Pensioner'), 25)
#Unemployed to 0     
app_df['Years Employed'] = app_df["Years Employed"].mask((app_df["Years Employed"] > 900) & (app_df['NAME_INCOME_TYPE'] == 'Unemployed'), 0)

In [ ]:
app_df['AMT_GOODS_PRICE'] = app_df['AMT_GOODS_PRICE'].fillna(app_df.groupby('Age')['AMT_GOODS_PRICE'].transform('mean')) #Changing base on age mean!

app_df['AMT_ANNUITY'] = app_df['AMT_ANNUITY'].fillna(app_df['AMT_ANNUITY'].mean())
app_df['CNT_FAM_MEMBERS_RANK'] = app_df['CNT_FAM_MEMBERS_RANK'].fillna(app_df['CNT_FAM_MEMBERS_RANK'].median())
app_df['DAYS_LAST_PHONE_CHANGE'] = app_df['DAYS_LAST_PHONE_CHANGE'].fillna(app_df['DAYS_LAST_PHONE_CHANGE'].mean())

In [ ]:
app_df['CODE_GENDER'].replace({'F':0, 'M':1, 'XNA': 0}, inplace=True)
app_df['NAME_CONTRACT_TYPE'].replace({'Cash loans':0, 'Revolving loans':1}, inplace=True)
app_df['FLAG_OWN_CAR'].replace({'N':0, 'Y':1}, inplace=True)
app_df['FLAG_OWN_REALTY'].replace({'Y':0, 'N':1}, inplace=True)

In [ ]:
# Making synthteic features for total income devided by annuity amount. same for credit amount:
app_df['INC_ANNUITY'] = app_df['AMT_INCOME_TOTAL'] / app_df['AMT_ANNUITY']
app_df['CRED_ANNUITY'] = app_df['AMT_CREDIT'] / app_df['AMT_ANNUITY']

In [ ]:
app_df = app_df.drop(labels=['DAYS_BIRTH', 'DAYS_EMPLOYED', 'BUILDING_TOTAL_AVG', 'CNT_CHILDREN', 'CNT_FAM_MEMBERS'], axis=1)

In [ ]:
# Replacing missing values to "Unaccompanied"
app_df['NAME_TYPE_SUITE'].replace(np.nan, 'Unaccompanied',regex=True, inplace=True)

# Replacing unknown to married since its the most common value
app_df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married',regex=True, inplace=True)

# It seems like it works better when changing some values before one hot enocoding all of them.
app_df['NAME_INCOME_TYPE'].replace(['Businessman', 'Student', 'Unemployed', 'Maternity leave'], 'Working', inplace=True)

app_df['ORGANIZATION_TYPE'].replace(['Industry: type 8', 'Trade: type 5', 'Trade: type 4','Industry: type 13', 'Religion', 'Industry: type 10', 'Industry: type 6'], 'Other', inplace=True)

# Lets try and save occupation_type by changing all 30% missing values to "other". maybe it would be predictive.
app_df['OCCUPATION_TYPE'].replace(np.nan, 'Other',regex=True, inplace=True)

# dummies for all + deleting IMMEDIATELY original columns
app_df = pd.get_dummies(data=app_df, columns=['OCCUPATION_TYPE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE', 'ORGANIZATION_TYPE', 'NAME_FAMILY_STATUS', 'WEEKDAY_APPR_PROCESS_START', 'CNT_FAM_MEMBERS_RANK', 'CNT_CHILDREN_RANK'])

In [ ]:
loan_counter = pd.DataFrame(previous['SK_ID_CURR'].value_counts()).reset_index() # Creating small df base on id count from previous df
loan_counter.columns = ['SK_ID_CURR', 'Total Loan Count'] #Adding names to counter

refused_df = pd.DataFrame(previous[previous['NAME_CONTRACT_STATUS'] == 'Refused'])
refused_counter = pd.DataFrame(refused_df['SK_ID_CURR'].value_counts()).reset_index() # Creating small df base on refused loan and id count from previous dfrefused_counter.columns = ['SK_ID_CURR', 'Refused Count'] #Adding names to counter
refused_counter.columns = ['SK_ID_CURR', 'Refused Count'] #Adding names to refused counter

approved_df = pd.DataFrame(previous[previous['NAME_CONTRACT_STATUS'] == 'Approved'])
approved_counter = pd.DataFrame(approved_df['SK_ID_CURR'].value_counts()).reset_index() # Creating small df base on approved loan and id count from previous dfrefused_counter.columns = ['SK_ID_CURR', 'Refused Count'] #Adding names to counter
approved_counter.columns = ['SK_ID_CURR', 'Approved Count'] #Adding names to approved counter

canceled_df = pd.DataFrame(previous[previous['NAME_CONTRACT_STATUS'] == 'Canceled'])
canceled_counter = pd.DataFrame(canceled_df['SK_ID_CURR'].value_counts()).reset_index() # Creating small df base on canceled loan and id count from previous dfrefused_counter.columns = ['SK_ID_CURR', 'Refused Count'] #Adding names to counter
canceled_counter.columns = ['SK_ID_CURR', 'Canceled Count'] #Adding names to canceled counter

unused_df = pd.DataFrame(previous[previous['NAME_CONTRACT_STATUS'] == 'Unused offer'])
unused_counter = pd.DataFrame(unused_df['SK_ID_CURR'].value_counts()).reset_index() # Creating small df base on un-used loan and id count from previous dfrefused_counter.columns = ['SK_ID_CURR', 'Refused Count'] #Adding names to counter
unused_counter.columns = ['SK_ID_CURR', 'Unused Count'] #Adding names to un-used counter

# Were going to add to app df a column that describes if client requested insurance in his last application.
# well take only the last by going to min "Days_Decision"
prev_insurance = previous.groupby('SK_ID_CURR', as_index=False).min('DAYS_DECISION')[['SK_ID_CURR', 'NFLAG_INSURED_ON_APPROVAL']]

# Next thing we're going to check in previous data for each example (ID) of current data, what is the minimun "DAYS_DECISION" from all values
# that the specific example in data has. that will be relative to the current application and therefore we make sure its last application per ID. 
previous['DAYS_DECISION'] = abs(previous['DAYS_DECISION']) # Converting and then adding -min- days decision to app_df
min_days_decision = previous.groupby('SK_ID_CURR', as_index=False).min('DAYS_DECISION')[['SK_ID_CURR', 'DAYS_DECISION']]

In [ ]:
app_df = app_df.join(loan_counter.set_index('SK_ID_CURR'), on='SK_ID_CURR') # Using pandas join function on id, to add loan count
app_df = app_df.join(refused_counter.set_index('SK_ID_CURR'), on='SK_ID_CURR') 
app_df = app_df.join(approved_counter.set_index('SK_ID_CURR'), on='SK_ID_CURR') 
app_df = app_df.join(canceled_counter.set_index('SK_ID_CURR'), on='SK_ID_CURR') 
app_df = app_df.join(unused_counter.set_index('SK_ID_CURR'), on='SK_ID_CURR') 

app_df = app_df.join(min_days_decision.set_index('SK_ID_CURR'), on='SK_ID_CURR') 
app_df = app_df.join(prev_insurance.set_index('SK_ID_CURR'), on='SK_ID_CURR')

In [ ]:
app_df.describe()

In [ ]:
app_df = app_df.drop(labels=['SK_ID_CURR', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR'], axis=1)

In [ ]:
target_names = ['Repayers', 'Defaulters']

In [ ]:
from sklearn.preprocessing import StandardScaler
app_df[['DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'HOUR_APPR_PROCESS_START', 'DAYS_LAST_PHONE_CHANGE', 'Age', 'Years Employed', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'Total Loan Count', 'Refused Count', 'Approved Count', 'Canceled Count', 'Unused Count', 'DAYS_DECISION', 'INC_ANNUITY', 'CRED_ANNUITY', 'NFLAG_INSURED_ON_APPROVAL']] = StandardScaler().fit_transform(app_df[['DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'HOUR_APPR_PROCESS_START', 'DAYS_LAST_PHONE_CHANGE', 'Age', 'Years Employed', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'Total Loan Count', 'Refused Count', 'Approved Count', 'Canceled Count', 'Unused Count', 'DAYS_DECISION', 'INC_ANNUITY', 'CRED_ANNUITY', 'NFLAG_INSURED_ON_APPROVAL']])

# Lets complete all nulls to 0 after we normazlied them. now all the nulls will have 0 values and wont effect distribution.
app_df.fillna({'Total Loan Count':0,'Refused Count':0, 'Approved Count':0, 'Unused Count':0, 'Canceled Count':0, 'DAYS_DECISION':0, 'NFLAG_INSURED_ON_APPROVAL':0, 'SQR DecisionEmployed':0},inplace=True)

In [ ]:
delete_outliers_columns = ['AMT_INCOME_TOTAL', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'Total Loan Count', 'Refused Count', 'Approved Count', 'Canceled Count', 'Unused Count']
for column in delete_outliers_columns:
    app_df = app_df[app_df[column] < 35]

In [ ]:
app_df.shape

# Predictions:
1. In all predictions we'll use 75% of data to train model and 25% for predictions
2. "Stratify=y" argument in train/test function causes that train and test will necessarily have same amount of defaulters (about 8%)
3. First prediction using all data to predict. Threshold can be changed in the final row in the cell that is before printing results
4. In the second prediction all variables splitted into 4 groups. In order to predict for specific groups, we can delete group or more from "X" in the 3rd line
5. In order to examine policies 8%/4%/2% change threshold by changing value of "predict_proba".
6. For 8% use 0.163-165.  For 4% use 0.199-0.201 . For 2% use 0.235-0.237.

In [ ]:
# Logistic regression for all data:
y = app_df["TARGET"]
X = app_df.drop('TARGET',axis=1)

# Split data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

# Prints statistics about TARGET:
print(pd.DataFrame(
        data=[
            ['y_TRAIN', y_train.shape, y_train.mean(), y_train.sum()],
            ['y_TEST', y_test.shape, y_test.mean(), y_test.sum()]
        ], 
        columns=['ID', 'SHAPE', 'MEAN Defaulters', 'SUM Defaulters']
    ),
    end='\n\n'
)


# Model predictions
logreg = LogisticRegression(penalty='l2', max_iter=1000)
logreg.fit(X_train, y_train)
Y_pred = (logreg.predict_proba(X_test)[:,1] >= 0.235).astype(int) #Threshold determines in this line

#Results
# Accuracy (less important result to examine)
print("accuracy score: ", accuracy_score(Y_pred, y_test))
# Confusion Matrix and Classification Report give more details about performance
print(classification_report(Y_pred, y_test, target_names=target_names, digits=4))

# Next we're going to split variables into 4 different groups

In [ ]:
#We can save more than one variable with out writing his name specificlly with "startswith" function

#group 1 - demographic:
demographic_indicators = [col for col in app_df if col.startswith(('CODE_GENDER', 'REGION', 'Age', 'ageX', 'NAME_EDUC', 'NAME_HOUSING', 'CNT', 'NAME_FAMILY'))]
#32 variables 

#group 2 - financial:
financial_indicators = [col for col in app_df if col.startswith(('AMT_INCOME_TOT', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'AMT_CREDIT', 'Years Emp', 'INC_AN', 'CRED_ANN','Total Loan', 'Refused', 'Approved', 'Canceled', 'Unused', 'DAYS_DECISION', 'NFLAG', 'OCCUPATION', 'NAME_INCOME', 'ORGANIZATION'))]
#88 variables

#group 3 - loan:
loan_indicators = [col for col in app_df if col.startswith(('NAME_CONTRACT', 'NAME_TYPE', 'AMT_ANNU', 'AMT_GOODS'))]
#10 variables

#group 4 - others:
other_indicators = [col for col in app_df if col.startswith(('DAYS_REG', 'DAYS_ID', 'DAYS_LAST', 'FLAG_MOBIL', 'FLAG_EMP', 'FLAG_WO', 'FLAG_CONT', 'FLAG_PHONE','FLAG_EMAIL', 'HOUR', 'FLAG_DOC', 'REG_REG', 'LIVE_REG', 'REG_CITY', 'LIVE_CITY', 'WEEKDAY'))]
#24 variables

# Predictions for groups of variables

In [ ]:
# Logistic regression using different groups of variables
y = app_df["TARGET"]
X = app_df[demographic_indicators + financial_indicators + loan_indicators + other_indicators]

# Split data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

# Prints statistics about TARGET:
print(pd.DataFrame(
        data=[
            ['y_TRAIN', y_train.shape, y_train.mean(), y_train.sum()],
            ['y_TEST', y_test.shape, y_test.mean(), y_test.sum()]
        ], 
        columns=['ID', 'SHAPE', 'MEAN Defaulters', 'SUM Defaulters']
    ),
    end='\n\n'
)

# Model predictions
logreg = LogisticRegression(penalty='l2', max_iter=1000)
logreg.fit(X_train, y_train)
Y_pred = (logreg.predict_proba(X_test)[:,1] >= 0.162).astype(int)

#Results
# Accuracy (less important result to examine)
print("accuracy score: ", accuracy_score(Y_pred, y_test))
# Confusion Matrix and Classification Report give more details about performance
print(classification_report(Y_pred, y_test, target_names=target_names, digits=4))